# Motivation

I have been working on independent projects for a long time now and have worked on projects that involve API integration, web scrapping, utilizing libraries etc. I was thinking of utilising my skills to develop a chatbot for  myself. But I realised that making a personal voice assistant would be even more fun. 
I came across speech recognition library and thought of giving it a try.

# What is a Voice Assistant?

A voice assistant or intelligent personal assistant is a software agent that can perform tasks or services for an individual based on verbal commands i.e. by interpreting human speech and respond via synthesized voices. Users can ask their assistants’ questions, control home automation devices, and media playback via voice, and manage other basic tasks such as email, to-do lists, open or close any application etc with verbal commands.

<img src="images/assistant.jpg">

# Future of Voice Assistants

Consumer awareness of voice devices and assistants, in terms of all the technology that exists and the capabilities that are already in place, is nascent. Yet, there’s no denying that voice is the future. The technology will continue to drive and shift consumer behavior, and companies need to prepare and adjust accordingly. Search, advertising, content, and commerce are being impacted industry-wide as consumers transform the way they interact with brands as the result of voice technology. Consumers are talking, and it’s a good idea to listen.

# Features of my own Voice Assistant

Thought the possibilities of a Voice Assistant are limited by our own creativity, we are going to build ours with following features: <br />
1) Getting the weather of our current location <br />
2) Top current news <br />
3) Your current location <br />
4) Tell you about things or people <br />
5) Play Youtube video <br />
6) Tell you a geeky joke

### Step-1 Importing the libraries

In [1]:
import speech_recognition as sr
import os
import sys
import re
import webbrowser
import requests
from pyowm import OWM
import urllib
import json
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import wikipediaapi
from pygeocoder import Geocoder
from geopy.geocoders import Nominatim
import geocoder
from webbrowser import open_new_tab
import pyjokes

### Step-2 Setting up voice input and output

In [2]:
#Getting voice input from the user
def myCommand():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print('''\nMy name is Adam, I can do these things for you:\nCurrent weather in your city\nLatest News\nYour location\nTell you about things & people\nPlay a song on Youtube\nTell a joke\n'''
             )
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=0.5)
        audio = r.listen(source)
    try:
        command = r.recognize_google(audio).lower()
        print('You said: ' + command + '\n')
    #loop back to continue to listen for commands if unrecognizable speech is received
    except sr.UnknownValueError:
        print('....')
        command = myCommand();
    return command
# Function to give voice output 
def response(audio):
    for line in audio.splitlines():
        os.system("say " + audio)

### Step-3 Adding functions to our Voice Assistant

In [3]:
def assistant(command):
    # current weather at your current location
    if 'weather' in command:
        f = open('/Users/ravidahiya/.secret/dark_api.json') #Loading my darksky api
        keys = json.load(f)
        api = keys['api_key']
        g = geocoder.ip('me') 
        latlong = g.latlng #Getting current lat-long
        weather = requests.get("https://api.darksky.net/forecast/{}/{},{}".format(api,latlong[0],latlong[1]))
        weather = weather.json()
        print('Current weather: {}\nCurrent temperature: {}\nHumidity: {}\nVisibility: {}'.format(
            weather['currently']['summary'],weather['currently']['temperature'],weather['currently']['humidity'],
            weather['currently']['visibility']))
    
    #Getting top current news from google news feed
    elif 'news' in command:
        news_url="https://news.google.com/news/rss"
        Client=urlopen(news_url)
        xml_page=Client.read()
        Client.close()
        soup_page=soup(xml_page,"xml")
        news_list=soup_page.findAll("item")
        # Print news title, url and publish date
        for news in news_list[0:5]:
            try:
                print(news.title.text)
                print(news.link.text)
                print(news.pubDate.text)
                response(news.title.text) #Read the news
                print("-"*60)
            except Exception as e:
                print(e)
                
    #Getting desired topics from wikipedia            
    elif 'tell me about' in command:
        reg_ex = re.search('tell me about (.*)', command) #to get the item to search in wikipedia
        wiki_wiki = wikipediaapi.Wikipedia('en')
        page = wiki_wiki.page(reg_ex.group(1))
        print(page.summary)
    
    #Getting current location
    elif 'where am i' in command or 'location' in command:
        g = geocoder.ip('me')
        latlong = g.latlng
        geolocator = Nominatim(user_agent="specify_your_app_name_here")
        location = geolocator.reverse(latlong)
        print(location.address)
    
    #Playing video on youtube
    elif 'play' in command:
        reg_ex = re.search('play (.*)', command)
        URL = ("https://www.google.com/search?btnI=I'm+Feeling+Lucky&ie=UTF-8"
       "&oe=UTF-8&q=site:youtube.com%20inurl:http://www.youtube.com/"
       "watch?v=%20{0}".format(reg_ex.group(1)))
        open_new_tab(URL)
    
    #Getting a random joke from pyjokes library
    elif 'joke' in command:
        joke = pyjokes.get_joke()
        print(joke)
        response(joke)
    
    #exit
    elif 'exit' in command or 'quit' in command:
        sys.exit(1)
    
    #to get next commnand from the user
    while True:
        assistant(myCommand())

In [4]:
assistant(myCommand())


My name is Adam, I can do these things for you:
Current weather in your city
Latest News
Your location
Tell you about things & people
Play a song on Youtube
Tell a joke

You said: what is the weather

Current weather: Mostly Cloudy
Current temperature: 44.23
Humidity: 0.88
Visibility: 10

My name is Adam, I can do these things for you:
Current weather in your city
Latest News
Your location
Tell you about things & people
Play a song on Youtube
Tell a joke

You said: tell me the news

Kazakhstan plane crash: At least 15 people killed after Bek Air flight carrying 98 passengers and crew crashes in Almaty - The Telegraph
https://news.google.com/__i/rss/rd/articles/CBMibmh0dHBzOi8vd3d3LnRlbGVncmFwaC5jby51ay9uZXdzLzIwMTkvMTIvMjcva2F6YWtoc3Rhbi1wbGFuZS1jcmFzaGJlay1haXItYWlyY3JhZnQtY2FycnlpbmctbmVhcmx5LTEwMC1wZW9wbGUv0gEA?oc=5
Fri, 27 Dec 2019 04:05:00 GMT
------------------------------------------------------------
Battersea fatal shooting: Flamur Beqiri 'may have had criminal links' - BBC N

SystemExit: 1

/Users/ravidahiya/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
